In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [100]:
df = pd.read_csv('sec_sentiment.csv', dtype='string')

In [101]:
del df['Unnamed: 0']

In [102]:
df['fdate'] = pd.to_datetime(df['fdate'])

In [103]:
df.Sentiment = df.Sentiment.astype(float)
df.lm_negative = df.lm_negative.astype(float)
df.lm_positive = df.lm_positive.astype(float)
df.lm_uncertainty = df.lm_uncertainty.astype(float)

In [104]:
df.lm_negative = df.groupby('TICKERH')['lm_negative'].transform(lambda v: v.ffill())
df.lm_positive = df.groupby('TICKERH')['lm_positive'].transform(lambda v: v.ffill())
df.lm_uncertainty = df.groupby('TICKERH')['lm_uncertainty'].transform(lambda v: v.ffill())

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548169 entries, 0 to 548168
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   fdate           548169 non-null  datetime64[ns]
 1   cik             6081 non-null    string        
 2   form            6081 non-null    string        
 3   lm_negative     548169 non-null  float64       
 4   lm_positive     548169 non-null  float64       
 5   lm_uncertainty  548169 non-null  float64       
 6   TICKERH         548169 non-null  string        
 7   Sentiment       548169 non-null  float64       
dtypes: datetime64[ns](1), float64(4), string(3)
memory usage: 33.5 MB


In [81]:
date_range = pd.date_range(df.fdate.min(), df.fdate.max(), freq='D')

In [82]:
full_df = pd.DataFrame({'fdate': list(date_range)}).merge(pd.DataFrame({'TICKERH': list(df.TICKERH.unique())}), how='cross')
full_df['neg'] = np.nan
full_df['pos'] = np.nan
full_df['unc'] = np.nan

In [83]:
for ticker in df.TICKERH.unique():
  ticker_df = df[df.TICKERH == ticker]
  ticker_df.index = pd.DatetimeIndex(ticker_df.fdate)
  ticker_df = ticker_df.reindex(date_range, fill_value=np.nan)
  full_df.loc[full_df.TICKERH == ticker, 'neg'] = ticker_df.lm_negative.ffill().values
  full_df.loc[full_df.TICKERH == ticker, 'pos'] = ticker_df.lm_positive.ffill().values
  full_df.loc[full_df.TICKERH == ticker, 'unc'] = ticker_df.lm_uncertainty.ffill().values

In [86]:
full_df.pivot(index='fdate', columns='TICKERH', values='neg').astype(float)

TICKERH,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMGN,AMT,AMZN,...,TXN,UNH,UNP,UPS,USB,V,VZ,WFC,WMT,XOM
fdate,,,,,,,,,,,,,,,,,,,,,
2008-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-21,0.010191,0.018307,0.014586,0.014191,0.017041,0.022489,0.014515,0.014763,0.013027,0.015964,...,0.011903,0.015414,0.015018,0.012448,0.018443,0.017339,0.012280,0.018263,0.016745,0.012859
2024-04-22,0.010191,0.018307,0.014586,0.014191,0.017041,0.022489,0.014515,0.014763,0.013027,0.015964,...,0.011903,0.015414,0.015018,0.012448,0.018443,0.017339,0.012280,0.018263,0.016745,0.012859
2024-04-23,0.010191,0.018307,0.014586,0.014191,0.017041,0.022489,0.014515,0.014763,0.013027,0.015964,...,0.011903,0.015414,0.015018,0.012448,0.018443,0.017339,0.012280,0.018263,0.016745,0.012859


In [106]:
t = full_df[pd.isnull(full_df.neg) & (full_df.fdate.dt.year > 2009)].TICKERH.unique()

In [107]:
t

array(['ABBV', 'AVGO', 'DIS', 'DOW', 'GM', 'GOOGL', 'KHC', 'MDT', 'META',
       'PYPL', 'TSLA'], dtype=object)

In [111]:
full_df[full_df.apply(lambda row: row.TICKERH not in t, axis=1)]

,fdate,TICKERH,neg,pos,unc
0,2008-01-09,AAPL,NaN,NaN,NaN
2,2008-01-09,ABT,NaN,NaN,NaN
3,2008-01-09,ACN,NaN,NaN,NaN
4,2008-01-09,ADBE,NaN,NaN,NaN
5,2008-01-09,AIG,NaN,NaN,NaN
...,...,...,...,...,...
589243,2024-04-25,V,0.018418,0.006579,0.006030
589244,2024-04-25,VZ,0.013114,0.006219,0.006279
589245,2024-04-25,WFC,0.018263,0.007695,0.007695
589246,2024-04-25,WMT,0.016745,0.005845,0.008490


In [96]:
df[pd.isnull(df.lm_negative) & (df.fdate.dt.year > 2009)].TICKERH.unique()

<StringArray>
[]
Length: 0, dtype: string

In [99]:
pd.isnull(df[df.fdate.dt.year > 2009].lm_negative).any()

False

In [57]:
aapl_df = df[df.TICKERH == 'AAPL']
aapl_df.index = pd.DatetimeIndex(aapl_df.fdate)
aapl_df = aapl_df.reindex(date_range, fill_value=np.nan)

In [58]:

full_df.loc[full_df.TICKERH == ticker, 'neg'] = aapl_df.lm_negative.ffill()
full_df.loc[full_df.TICKERH == ticker, 'pos'] = aapl_df.lm_positive.ffill()
full_df.loc[full_df.TICKERH == ticker, 'unc'] = aapl_df.lm_uncertainty.ffill()

In [60]:
aapl_df.lm_negative.ffill()

2008-01-09         NaN
2008-01-10         NaN
2008-01-11         NaN
2008-01-12         NaN
2008-01-13         NaN
                ...   
2024-04-21    0.010191
2024-04-22    0.010191
2024-04-23    0.010191
2024-04-24    0.010191
2024-04-25    0.010191
Freq: D, Name: lm_negative, Length: 5952, dtype: float64

In [62]:
full_df.loc[full_df.TICKERH == 'AAPL', 'neg'] = aapl_df.lm_negative.ffill()

In [79]:
full_df.loc[full_df.TICKERH == 'AAPL', 'neg'] = aapl_df.lm_negative.ffill().values

In [80]:
full_df.loc[full_df.TICKERH == 'AAPL', 'neg']

0              NaN
99             NaN
198            NaN
297            NaN
396            NaN
            ...   
588753    0.010191
588852    0.010191
588951    0.010191
589050    0.010191
589149    0.010191
Name: neg, Length: 5952, dtype: float64

In [66]:
len(full_df.loc[full_df.TICKERH == 'AAPL', 'neg'])

5952

In [67]:
len(aapl_df.lm_negative.ffill())

5952

In [2]:
# read all data
df = pd.read_csv('crsp_snp100_2010_to_2024.csv', dtype='string')
sec_df = pd.read_csv('sec_sentiment.csv', dtype='string')

# convert datatypes for CRSP data
df = df[['date', 'TICKER', 'PRC', 'VOL', 'ASKHI', 'BIDLO', 'FACPR']]
df.date = pd.to_datetime(df.date)
df.FACPR = df.FACPR.fillna('0.0')
df.astype({
    'PRC': float,
    'VOL': float,
    'ASKHI': float,
    'BIDLO': float,
    'FACPR': float
})
df = df.drop_duplicates(subset=['date', 'TICKER'])
df.dropna(inplace=True)

# clean sentiment data
del sec_df['Unnamed: 0']
sec_df['fdate'] = pd.to_datetime(sec_df['fdate'])
sec_df.Sentiment = sec_df.Sentiment.astype(float)
sec_df.lm_negative = sec_df.lm_negative.astype(float)
sec_df.lm_positive = sec_df.lm_positive.astype(float)
sec_df.lm_uncertainty = sec_df.lm_uncertainty.astype(float)
sec_df.lm_negative = sec_df.groupby('TICKERH')['lm_negative'].transform(lambda v: v.ffill())
sec_df.lm_positive = sec_df.groupby('TICKERH')['lm_positive'].transform(lambda v: v.ffill())
sec_df.lm_uncertainty = sec_df.groupby('TICKERH')['lm_uncertainty'].transform(lambda v: v.ffill())

# fill in missing dates for sentiment
date_range = pd.date_range(sec_df.fdate.min(), sec_df.fdate.max(), freq='D')
full_df = pd.DataFrame({'fdate': list(date_range)}).merge(pd.DataFrame({'TICKERH': list(sec_df.TICKERH.unique())}), how='cross')
full_df['neg'] = np.nan
full_df['pos'] = np.nan
full_df['unc'] = np.nan
for ticker in sec_df.TICKERH.unique():
    ticker_df = sec_df[sec_df.TICKERH == ticker]
    ticker_df.index = pd.DatetimeIndex(ticker_df.fdate)
    ticker_df = ticker_df.reindex(date_range, fill_value=np.nan)
    full_df.loc[full_df.TICKERH == ticker, 'neg'] = ticker_df.lm_negative.ffill().values
    full_df.loc[full_df.TICKERH == ticker, 'pos'] = ticker_df.lm_positive.ffill().values
    full_df.loc[full_df.TICKERH == ticker, 'unc'] = ticker_df.lm_uncertainty.ffill().values

# only include stocks that are present in all dates and in both datasets
df_ticker_ok = df.TICKER.value_counts() == df.TICKER.value_counts().max()
not_in_sent = full_df[pd.isnull(full_df.neg) & (full_df.fdate.dt.year > 2009)].TICKERH.unique()
def is_max_val_count(ticker: str) -> bool:
  return df_ticker_ok[ticker] and (ticker not in ['GOOG', 'EXC']) and (ticker not in not_in_sent)
df = df[df.apply(lambda row: is_max_val_count(row['TICKER']), axis=1)]
df = df[(df.date.dt.year >= 2010) & (df.date.dt.year <= 2019)]
times = df.date.unique()[1:]
tickers = df.TICKER.unique()
full_df = full_df[full_df.apply(lambda row: row.TICKERH not in tickers, axis=1)]

# create stock array
stock_df = df.pivot(index='date', columns='TICKER', values='PRC').astype(float)
high_df = df.pivot(index='date', columns='TICKER', values='ASKHI').astype(float)
low_df = df.pivot(index='date', columns='TICKER', values='BIDLO').astype(float)

# adjust for stock splits
facpr_df = df.pivot(index='date', columns='TICKER', values='FACPR').astype(float)
stock_df = stock_df * (1+facpr_df).cumprod(axis=0)
high_df = high_df * (1+facpr_df).cumprod(axis=0)
low_df = low_df * (1+facpr_df).cumprod(axis=0)
ret = np.log(stock_df.pct_change().iloc[1:, :] + 1)

# create pivot tables
neg_sent_df = full_df.pivot(index='fdate', columns='TICKERH', values='neg').astype(float)
pos_sent_df = full_df.pivot(index='fdate', columns='TICKERH', values='pos').astype(float)
unc_sent_df = full_df.pivot(index='fdate', columns='TICKERH', values='unc').astype(float)

num_time_periods = len(times)-15-1
universe_size = len(tickers)

In [3]:
t = 100

In [6]:
neg_sent_df.loc[times[t:t+15], :]

TICKERH,ABBV,AVGO,BKNG,C,CHTR,DIS,DOW,GM,GOOGL,INTU,KHC,LIN,MDLZ,MDT,META,NEE,PYPL,RTX,TMUS,TSLA
fdate,,,,,,,,,,,,,,,,,,,,
2010-05-28,NaN,NaN,0.029978,0.078428,0.023379,NaN,NaN,0.018786,NaN,0.118088,NaN,0.015099,0.103683,NaN,NaN,0.016461,NaN,0.078085,0.023393,NaN
2010-06-01,NaN,NaN,0.029978,0.078428,0.023379,NaN,NaN,0.018786,NaN,0.118088,NaN,0.015099,0.103683,NaN,NaN,0.016461,NaN,0.078085,0.023393,NaN
2010-06-02,NaN,NaN,0.029978,0.078428,0.023379,NaN,NaN,0.018786,NaN,0.118088,NaN,0.015099,0.103683,NaN,NaN,0.016461,NaN,0.078085,0.023393,NaN
2010-06-03,NaN,NaN,0.029978,0.078428,0.023379,NaN,NaN,0.018786,NaN,0.118088,NaN,0.015099,0.103683,NaN,NaN,0.016461,NaN,0.078085,0.023393,NaN
2010-06-04,NaN,NaN,0.029978,0.078428,0.023379,NaN,NaN,0.018786,NaN,0.118088,NaN,0.015099,0.103683,NaN,NaN,0.016461,NaN,0.078085,0.023393,NaN
2010-06-07,NaN,NaN,0.029978,0.078428,0.023379,NaN,NaN,0.018786,NaN,0.118088,NaN,0.015099,0.103683,NaN,NaN,0.016461,NaN,0.078085,0.023393,NaN
2010-06-08,NaN,NaN,0.029978,0.078428,0.023379,NaN,NaN,0.018786,NaN,0.118088,NaN,0.015099,0.103683,NaN,NaN,0.016461,NaN,0.078085,0.023393,NaN
2010-06-09,NaN,NaN,0.029978,0.078428,0.023379,NaN,NaN,0.018786,NaN,0.118088,NaN,0.015099,0.103683,NaN,NaN,0.016461,NaN,0.078085,0.023393,NaN
2010-06-10,NaN,NaN,0.029978,0.078428,0.023379,NaN,NaN,0.018786,NaN,0.118088,NaN,0.015099,0.103683,NaN,NaN,0.016461,NaN,0.078085,0.023393,NaN


In [4]:
s = np.zeros((6, universe_size, 10))
s[0, :, :] = stock_df.loc[times[t:t+10], :].to_numpy().T
s[1, :, :] = high_df.loc[times[t:t+10], :].to_numpy().T
s[2, :, :] = low_df.loc[times[t:t+10], :].to_numpy().T
s[3, :, :] = neg_sent_df.loc[times[t:t+10], tickers].to_numpy().T
s[4, :, :] = pos_sent_df.loc[times[t:t+10], tickers].to_numpy().T
s[5, :, :] = unc_sent_df.loc[times[t:t+10], tickers].to_numpy().T

KeyError: "None of [Index(['ORCL', 'MSFT', 'HON', 'KO', 'XOM', 'GD', 'GE', 'IBM', 'PEP', 'MO',\n       'COP', 'AMGN', 'CVX', 'AAPL', 'TXN', 'CVS', 'PG', 'SO', 'CAT', 'CL',\n       'DE', 'BMY', 'BA', 'ABT', 'LMT', 'PFE', 'EMR', 'JNJ', 'MMM', 'MRK', 'F',\n       'DUK', 'WFC', 'MCD', 'JPM', 'UNP', 'TGT', 'BK', 'DHR', 'LLY', 'WMT',\n       'NKE', 'AXP', 'INTC', 'BAC', 'FDX', 'AMD', 'LOW', 'TMO', 'VZ', 'T',\n       'USB', 'HD', 'AIG', 'MS', 'SCHW', 'ADBE', 'CSCO', 'QCOM', 'GILD',\n       'SBUX', 'SPG', 'COF', 'AMZN', 'AMT', 'NVDA', 'GS', 'COST', 'BLK', 'UPS',\n       'MET', 'ACN', 'NFLX', 'CMCSA', 'CRM', 'MA', 'PM', 'V', 'UNH'],\n      dtype='object', name='TICKERH')] are in the [columns]"